# Let's bring up how many files do we have

In [1]:
import os
from skelcast.data.prepare_data import get_missing_files, get_skeleton_files, filter_missing

DATA_DIR = os.environ['NTU_RGBD_DATA_DIR']
MISSING_FILES_DIR = '../data/missing'

if not DATA_DIR:
    raise ValueError("NTU_RGBD_DATA_DIR environment variable is not set or empty.")

missing_skel_files = get_missing_files(MISSING_FILES_DIR)
print(f'Missing skeleton files: {len(missing_skel_files)}')
skeleton_files = get_skeleton_files(DATA_DIR)

print(f"Found {len(skeleton_files)} .skeleton files.")

skeleton_files_filtered = filter_missing(skeleton_files=skeleton_files,
                                         missing_skeleton_names=missing_skel_files)

nturgb_d.txt
nturgb_d_120.txt
Missing skeleton files: 837
Found 114480 .skeleton files.
Skeleton files after filtering: 114478 files left.


In [12]:
import numpy as np

from skelcast.data.dataset import read_skeleton_file

mat = read_skeleton_file(file_path=skeleton_files_filtered[884])

mat.get('skel_body0').shape

(23, 25, 3)

In [13]:
durations = []
for skel_file in skeleton_files_filtered:
    mat = read_skeleton_file(skel_file)
    joints = mat.get('skel_body0')
    duration = joints.shape[0]
    durations.append(duration)

ValueError: max() arg is an empty sequence